# Imports

In [ ]:
# Data manipulation and analysis
import pandas as pd
import numpy as np

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Deep learning with TensorFlow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical

# Deep learning with PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision import models

# Natural Language Processing (NLP)
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Miscellaneous
import os
import re
import time
import pickle
import cv2

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Getting Audio Clips of CREMA-D

In [ ]:
!pip install moviepy
from moviepy.editor import VideoFileClip

In [ ]:
directory = "/content/drive/My Drive/Machine Learning/COSMOS/FINAL_PROJECT/DER/CREMA-D-Videos/"
to_directory = "/content/drive/My Drive/Machine Learning/COSMOS/FINAL_PROJECT/DER/CD-Audios/"

edict = { # this matches up with the indices for Ekman-6
    'ANG': 0,
    'DIS': 1,
    'FEA': 2,
    'HAP': 3,
    'SAD': 4,
}

id = 250
files = [[], [], [], [], []]

for file in os.listdir(directory):
  filename = os.fsdecode(file)
  emote = filename[9:12]
  if emote in edict:
    files[edict[emote]].append(filename)

def write(read_path, write_path):
  clip = VideoFileClip(read_path)
  audio = clip.audio
  audio.write_audiofile(write_path)
  audio.close()
  clip.close()

for i in range(5):
  for j in range(50):
    read_path = directory + files[i][j]
    write_path = to_directory + str(id) + ".mp3"
    write(read_path, write_path)
    id += 1

# Speech to Text

In [ ]:
!pip install openai

In [ ]:
from openai import OpenAI
key = "sk-proj-WWodOl2OF1TJU7otVteuT3BlbkFJGeMH2FCoSxPCSXj5aNMt"
client = OpenAI(api_key = key)

def translated_text(filename):
  audio_file = open(filename, "rb")
  translation = client.audio.translations.create(
    model="whisper-1",
    file=audio_file,
    temperature = 0.90
  )
  return translation.text

texts = []

for index in range(250, 500):
  texts.append(translated_text(to_directory + str(index) + ".mp3"))

In [ ]:
import csv

with open("/content/drive/My Drive/Machine Learning/COSMOS/FINAL_PROJECT/DER/CD_texts/transcripts.csv", 'w') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(texts)

# Image to Spectrogram

# Frame Cutting